In [1]:
from base import *
from utils import pauli_sum, entropy
from plot_tools import *
import plot_utils
import qiskit as qk
from qiskit import QuantumCircuit, Aer, execute
from qiskit.primitives import Estimator
# from qiskit.utils import QuantumInstance
# from qiskit.algorithms import VQE
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP, ADAM, COBYLA
from qiskit.opflow import X, Z, I, Y, AerPauliExpectation, PauliSumOp
from qiskit.circuit import Parameter
import numpy as np
import seaborn as sns



## The Two qubit systems
$$
H_{\mathrm{I}}=H_x\sigma_x\otimes\sigma_x+H_z\sigma_z\otimes\sigma_z,
$$

In [2]:
# Constant
Hx = 2.0
Hz = 3.0

# H_0
H0_coeff = np.array([0.0, 2.5, 6.5, 7.0])
H0 = np.diag(H0_coeff)


H1 = np.array([[Hz, 0, 0, 0],
            [0, -Hz, Hx, 0],
            [0,  Hx, -Hz, 0],
            [Hx, 0, 0, + Hz]])
H0

array([[0. , 0. , 0. , 0. ],
       [0. , 2.5, 0. , 0. ],
       [0. , 0. , 6.5, 0. ],
       [0. , 0. , 0. , 7. ]])

### Classical numerical solver for the eigenvalues and the Von Neumann entropy 

$$
S(A,B)=-\mathrm{Tr}\left(\rho_{A,B}\log_2 (\rho_{A,B})\right).
$$

In [78]:
def H(lmb):
    return H0 + lmb * H1

# def partial_trace(state, i):
'''deprecated, replaced by np.einsum'''
#     state0 = np.array([1,0])
#     state1 = np.array([0,1])
#     density = np.outer(state, np.conj(state))
#     op0 = np.kron(state0, I) if i == 0 else np.kron(I, state0)
#     op1 = np.kron(state1, I) if i == 0 else np.kron(I, state1)
#     return op0.conj() @ density @ op0.T + op1.conj() @ density @ op1.T



lmbs = np.arange(0, 1, 0.01)
eigs = np.zeros((4, len(lmbs)))
entropies = np.zeros((4, len(lmbs)))
vn_entropies = np.zeros((4, len(lmbs)))

for i, lmb in enumerate(lmbs):
    ham = H(lmb)
    eigvals, eigvecs = np.linalg.eig(ham)
    eigs[:,i] = sorted(eigvals)
    lmb_entropy = np.zeros(4)
    vn_entropies[:,i] = entropy(ham, np.arange(4))[0] # returns both, doesn't matter which one you take
    # vn_entropies[:,i] = entropy(ham, 0)[0]

vn_entropies.shape

(4, 100)

In [80]:
plot_2D(lmbs, eigs, plot_count=4, title=r"Numerical Eigenvalues for 2 Qubit system vs $\lambda$", x_title=r'$\lambda$', y_title='Energies',label=[r'$E_0$', r'$E_1$', r'$E_2$', r'$E_3$'], filename='eigs-2-cl.pdf')
plot_2D(lmbs, vn_entropies[0], plot_count=1, label=[f"Entropy {i}" for i in range(4)],title="Entropies", x_title=r'$\lambda$', y_title='Entropies',filename='entropies-2-0.pdf')

### VQE

First we need to rewrite the hamiltonian in terms of Pauli matrices.
$$ H_0 = \begin{bmatrix}
    0 & 0 & 0 & 0  \\
    0 & 2.5 & 0 & 0  \\
    0 & 0 & 6.5 & 0 \\
    0 & 0 & 0 & 7 
    \end{bmatrix}
$$

Rewrite $H_0$ in terms of the computational basis $Z$ and $I$.
$$
    H_0 = a I\otimes I + b I\otimes Z + cZ\otimes I + dZ\otimes Z,
$$
Solving the above equations, we get
$$
a = 4, b = -0.75, c = -2.75, d = -0.5
$$

and using the gates we have to switch to the computational basis $Z \otimes I$ we have:

Let the swap gate be denoted by $\mathcal{S}$, and cnot gate with control on qubit 1 and target on qubit 0 be $\mathcal{CN}$.
$$
\newcommand{\swap}{\mathcal{S}} 
\newcommand{\cnot}{\mathcal{CN}(1,0)} 
$$
$$ 
H_0 = 4 I\otimes I + -0.75 (\swap Z \otimes I \swap) + -2.75 Z\otimes I + -0.5 (\cnot Z \otimes I \cnot) 
$$

$$
H_{\mathrm{I}}=H_x\sigma_x\otimes\sigma_x+H_z\sigma_z\otimes\sigma_z,
$$H
Rewrite $X$ into $I \otimes Z$
$$
H_I = H_x (I \otimes \sigma_z) (\sigma_z \otimes I) + H_z (\sigma_z \otimes I) (I \otimes \sigma_z)
$$

$$
H_I = \lambda (Hx \swap Z \otimes I \swap) + \lambda (Hz \cnot Z \otimes I \cnot)
$$

$$
\begin{aligned}
H = H_0 + H_I = &4 \: I\otimes I \\
&+ (-0.75 + \lambda Hx) \: (\swap Z \otimes I \swap) \\
&+ -2.75 \: Z \otimes I \\
&+ (-0.5 + \lambda Hz) \:(\cnot Z \otimes I \cnot) \\
\end{aligned}


In [5]:
def H(lmb): 
    return PauliSumOp.from_list([('II', 4),
                                 ('IZ', (-0.75)),
                                 ('ZI', (-2.75)),
                                 ('ZZ', (-0.5 + lmb * Hz)),
                                 ('XX', (lmb * Hx))])

H(0)


PauliSumOp(SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ', 'XX'],
              coeffs=[ 4.  +0.j, -0.75+0.j, -2.75+0.j, -0.5 +0.j,  0.  +0.j]), coeff=1.0)

Set up

In [27]:
def ansatz():
    '''creates an ansatz for vqe
    inputs:
        n: int, number of qubits
    return:
        qc: QuantumCircuit object, parametrized quantum circuit with given number of qubits, with parameters 
        thetas and phis
        '''
    # TODO: maybe can be changed to Hartree Fock initial state? 
    # from HartreeFock.construct_circuit()
    n=2
    qc = QuantumCircuit(n)
    theta0 = Parameter('theta0')
    phi0 = Parameter('phi0')
    theta1 = Parameter('theta1')
    phi1 = Parameter('phi1')

    qc.rx(theta0,0)
    qc.ry(phi0,0)
    qc.rx(theta1,1)
    qc.ry(phi1,1)
    qc.cx(0,1)
    
    return qc

def prep_vqe(n):
    '''input:
        n: int, number of qubits'''
    adam = ADAM(maxiter = 10000)
    slsqp = SLSQP(maxiter = 10000)
    cobyla = COBYLA(maxiter = 500)
    # backend = Aer.get_backend('qasm_simulator')
    estimator = Estimator()

    vqe = VQE(ansatz=ansatz(), 
              optimizer=slsqp, 
              initial_point=np.random.uniform(low=-np.pi, high=np.pi, size=2*n), 
              estimator=estimator)

    return vqe
# vqe is sucessful becuase we enbed 2 qubit gates which, which creates highly entangled states. 
# Allowing us to explore multiple states at the same time. thetas needed for an n_qubit state doesn't have to scale exponentially with the number of qubits, but only linearly.



In [37]:
n_points=100
n_qubits=2

lmbs = np.arange(n_points) # rmb to divide when used
vqe = prep_vqe(n_qubits)
energies = np.zeros(n_points) # containers for the energies

for i, lmb in enumerate(lmbs):
    lmb = lmb / n_points
    ham = H(lmb)
    # print(ham)
    result = vqe.compute_minimum_eigenvalue(ham)
    energies[i] = result.eigenvalue.real
    # print(f'lmb = {lmb}, energy = {result.eigenvalue.real}')




In [39]:
# THIS ONE IS WILD
plt.plot(lmbs/n_points, energies)
# plot_2D(lmbs/n_points, energies, title="Ground State Energy for 2 qubits with qiskit", x_title=r'$\lambda$', y_title='Energy',
        # filename='qk-2.pdf'), 
save_fig('qk-1qb.pdf')

plt.show()

With **src.base**
$$
\begin{aligned}
H = H_0 + H_I = &4 \: I\otimes I \\
&+ (-0.75 ) \: (\swap Z \otimes I \swap) \\
&+ -2.75 \: Z \otimes I \\
&+ (-0.5 + \lambda Hz) \:(\cnot Z \otimes I \cnot) \\
&+ \lambda Hx \: (H \otimes H \cnot Z \otimes I \cnot H \otimes H)
\end{aligned}

In [41]:

def my_ansatz(params):
    '''params: list of parameters for the ansatz, [thetas, phis]'''
    qc = Qubits_2()
    qc.rx(params[0], 0)
    qc.ry(params[1], 0)
    qc.rx(params[2], 1)
    qc.ry(params[3], 1)
    qc.cnot(0, 1)
    return qc

def my_H(lmb):
    return pauli_sum([('II', 4),
            ('IZ', (-0.75)),
            ('ZI', (-2.75)),
            ('ZZ', (-0.5 + lmb * Hz)),
            ('XX', (lmb * Hx))])

def expectation(qc, lmb, n_shots=10000):
    '''the expectation value of energy for the hamiltonian'''
    coeffs = {'II':4, 'IZ':-0.75, 'ZI':-2.75, 'ZZ': -0.5 + lmb * Hz, 'XX':lmb * Hx}
    qc_copy = qc.copy()

    # IZ
    qc_copy.swap() 
    count_iz = qc_copy.measure(n_shots=n_shots)[:,0] # only the couonts, don't keep the states

    # ZI
    qc_copy = qc.copy() # make a new copy everytime
    count_zi = qc_copy.measure(n_shots=n_shots)[:,0]

    # ZZ
    qc_copy = qc.copy()
    qc_copy.cnot(1,0)
    count_zz = qc_copy.measure(n_shots=n_shots)[:,0]

    # XX
    qc_copy = qc.copy()
    qc_copy.hadamard(0)
    qc_copy.hadamard(1)
    qc_copy.cnot(1,0)

    count_xx = qc_copy.measure(n_shots=n_shots)[:,0]

    expectaions = 0
    # ZI = (#00 + #01 - #10 - #11)/ number of shots
    # print(count_iz, count_zi, count_zz, count_xx)
    counts = [coeffs['IZ'] * count_iz, coeffs['ZI'] * count_zi, 
              coeffs['ZZ'] * count_zz, coeffs['XX'] * count_xx]
    # print(counts)
    for count in counts:
        # print(count)
        for i, state_count in enumerate(count):
            if i < 2:
                expectaions += state_count
            else:
                expectaions -= state_count
    return coeffs['II'] + expectaions / n_shots


qc = Qubits_2()
expectation(qc, 0.5)


1.4908000000000001

In [42]:
from algorithms import Vqe 

In [48]:
n_points = 100

init_points = np.random.uniform(low=-np.pi, high=np.pi, size=4)
my_vqe = Vqe(my_ansatz, init_points, expectation=expectation)

my_energies = np.zeros(n_points)
lmbs = np.arange(n_points)
for i, lmb in enumerate(lmbs):
    lmb = lmb / n_points
    my_ham = my_H(lmb)
    min_params, min_energy = my_vqe.minimise_eigenvalue(my_ham, lmb, 1000)
    # print(f"lambda: {lmb}, energy: {min_energy}")
    my_energies[i] = min_energy



In [50]:
lmbs = lmbs/n_points
plot_2D(lmbs, [my_energies,energies, eigs[0]], plot_count=3, title='Ground State Energy Solver Comparison', x_title=r'$\lambda$', y_title='Eigenvalues', label=['vqe-base','vqe-qk', 'numerical'],
        filename='2qb-all.pdf')
